In [28]:
from google.cloud import bigquery
import pandas

In [29]:
!gsutil ls gs://mysql_table_csv_9287/csv

gs://mysql_table_csv_9287/csv/img_path.csv
gs://mysql_table_csv_9287/csv/img_urls.csv
gs://mysql_table_csv_9287/csv/img_wikidata_id.csv
gs://mysql_table_csv_9287/csv/names.csv


In [30]:
location = "asia-northeast1"
client = bigquery.Client(location=location, project="root-beanbag-354111")
print("Client creating using default project: {}".format(client.project))

Client creating using default project: root-beanbag-354111


In [31]:
# transfer mysql database to bq via csv in gcs

# Define a name for the new dataset.
dataset_id = 'scraping_dog_breeds_by_name'

# delete dataset used before
dataset = client.delete_dataset(dataset_id, delete_contents = True, not_found_ok = True)

# The project defaults to the Client's project if not specified.
dataset = client.create_dataset(dataset_id)  # API request

In [32]:
def loadDataset(schema, name_table):
    schema = [bigquery.SchemaField(c, t) for (c, t) in schema]
    # Configure the load job
    job_config = bigquery.LoadJobConfig(
        schema=schema,
        skip_leading_rows=1,
        # The source format defaults to CSV. The line below is optional.
        source_format=bigquery.SourceFormat.CSV
    )

    # img_path.csv to img_path table
    uri = 'gs://mysql_table_csv_9287/csv/' + name_table + '.csv'
    destination_table_ref = dataset.table(name_table)

    # Start the load job
    load_job = client.load_table_from_uri(
        uri, destination_table_ref, job_config=job_config)
    print('Starting job {}'.format(load_job.job_id))

    load_job.result()  # Waits for table load to complete.
    print('Job finished.')

    # Retreive the destination table
    destination_table = client.get_table(destination_table_ref)
    print('Loaded {} rows.'.format(destination_table.num_rows))

In [33]:
schema = [('wikidata_id', 'STRING'),('name', 'STRING')]
loadDataset(schema, "names")

schema = [('img_id', 'INT64'),('wikidata_id', 'STRING')]
loadDataset(schema, "img_wikidata_id")

schema = [('img_id', 'INT64'),('path', 'STRING')]
loadDataset(schema, "img_path")

Starting job 41f9b149-e96e-4c42-9662-56c5de58133f
Job finished.
Loaded 520 rows.
Starting job f1806d84-81b2-47ee-9eab-188cd6820745
Job finished.
Loaded 23047 rows.
Starting job a2b45463-c4e7-421a-a5e5-91ad723852f5
Job finished.
Loaded 23047 rows.


In [34]:
!bq ls scraping_dog_breeds_by_name

      tableId       Type    Labels   Time Partitioning   Clustered Fields  
 ----------------- ------- -------- ------------------- ------------------ 
  img_path          TABLE                                                  
  img_wikidata_id   TABLE                                                  
  names             TABLE                                                  


In [35]:
def query2df(query):
    query = query
    query_job = client.query(
        query,
        # Location must match that of the dataset(s) referenced in the query.
        location=location
    )  # API request - starts the query

    df = query_job.to_dataframe()
    return df

In [36]:
query = """
    SELECT *
    FROM `root-beanbag-354111.scraping_dog_breeds_by_name.names`
    LIMIT 10
"""
query2df(query)

,wikidata_id,name
0,Q10345998,Berger des Pyrénées à poil long
1,Q10520346,Greek Shepherd Dog
2,Q105955,Elo (dog breed)
3,Q10657975,Russian Hound
4,Q107303989,Buckhound
5,Q107448399,West Country Harrier
6,Q107804694,Sarail hound
7,Q1092646,Ciobanesc Romanesc Carpatin
8,Q11044643,Swinford Bandog
9,Q1118460,Shiba Inu


In [39]:
query = """
    SELECT *
    FROM `root-beanbag-354111.scraping_dog_breeds_by_name.img_wikidata_id`
    LIMIT 10
"""
query2df(query)

,img_id,wikidata_id
0,3351,Q10345998
1,3352,Q10345998
2,3353,Q10345998
3,3354,Q10345998
4,3355,Q10345998
5,3356,Q10345998
6,3357,Q10345998
7,3358,Q10345998
8,3359,Q10345998
9,3360,Q10345998


In [38]:
query = """
    SELECT *
    FROM `root-beanbag-354111.scraping_dog_breeds_by_name.img_path`
    LIMIT 10
"""
query2df(query)

,img_id,path
0,1,./imgs/Q7254/image_0000.jpg
1,2,./imgs/Q7254/image_0001.jpg
2,3,./imgs/Q7254/image_0002.jpg
3,4,./imgs/Q7254/image_0003.jpg
4,5,./imgs/Q7254/image_0004.jpg
5,6,./imgs/Q7254/image_0005.jpg
6,7,./imgs/Q7254/image_0006.jpg
7,8,./imgs/Q7254/image_0007.jpg
8,9,./imgs/Q7254/image_0008.jpg
9,10,./imgs/Q7254/image_0009.jpg


In [44]:
# join all tables and output for ml pre-processing

query = """
    SELECT ip.img_id, ip.path, iw.wikidata_id, n.name
    FROM `root-beanbag-354111.scraping_dog_breeds_by_name.img_path` AS ip
    INNER JOIN `root-beanbag-354111.scraping_dog_breeds_by_name.img_wikidata_id` AS iw
    ON ip.img_id = iw.img_id
    INNER JOIN `root-beanbag-354111.scraping_dog_breeds_by_name.names` AS n
    ON iw.wikidata_id = n.wikidata_id
    LIMIT 10
"""
query2df(query)

,img_id,path,wikidata_id,name
0,1,./imgs/Q7254/image_0000.jpg,Q7254,Affenpinscher
1,2,./imgs/Q7254/image_0001.jpg,Q7254,Affenpinscher
2,3,./imgs/Q7254/image_0002.jpg,Q7254,Affenpinscher
3,4,./imgs/Q7254/image_0003.jpg,Q7254,Affenpinscher
4,5,./imgs/Q7254/image_0004.jpg,Q7254,Affenpinscher
5,6,./imgs/Q7254/image_0005.jpg,Q7254,Affenpinscher
6,7,./imgs/Q7254/image_0006.jpg,Q7254,Affenpinscher
7,8,./imgs/Q7254/image_0007.jpg,Q7254,Affenpinscher
8,9,./imgs/Q7254/image_0008.jpg,Q7254,Affenpinscher
9,10,./imgs/Q7254/image_0009.jpg,Q7254,Affenpinscher
